In [1]:
% matplotlib inline

/Users/kei/penv2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.externals import joblib

In [3]:
advertiserlist_path = '../data/adtech_intern_advertiserlist.csv'
log_path = '../data/intern_samplelog.csv'

In [4]:
advertiserlist = pd.read_csv(advertiserlist_path)
log_original = pd.read_csv(log_path)
log = log_original

In [5]:
log.loc[:,'os'][log.os=='iOS'] = 0
log.loc[:,'os'][log.os=='Android'] = 1

/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [6]:
def hashnum(modnum, data):
    mod = 10 ** modnum
    return hash(data) % mod

In [7]:
logEachAd = [log[log.advertiser==ad].drop('advertiser', axis=1)for ad in xrange(1, 11)]
input_type='string'
param = {'max_depth':6, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2
modnum = 3

for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os]
    dlog = xgb.DMatrix(log_data,label=log_target)
    print xgb.cv(param, dlog, num_round, nfold=5, metrics={'auc'}, seed = 0)
    xgb_model = xgb.train(param, dlog, num_round)
    model_path = '../models/Ad%d_xgb.pkl'%(ind+1)
    joblib.dump(xgb_model, model_path)

----------------- Advertiser 1 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.649815      0.004170        0.652411       0.000263
1       0.658968      0.002833        0.663175       0.002709
----------------- Advertiser 2 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.556518      0.005174        0.564538       0.004712
1       0.560640      0.003240        0.572066       0.003214
----------------- Advertiser 3 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.591840      0.002883        0.594412        0.00078
1       0.616277      0.002809        0.622320        0.00089
----------------- Advertiser 4 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.552416      0.004076        0.555671       0.002241
1       0.583160      0.004819        0.589943       0.003873
----------------- Advertiser 5 -------------------
 

In [12]:
xgb_models = [joblib.load('../models/Ad%d_xgb.pkl'%ind) for ind in xrange(1,11)]

In [13]:
test_sample = log.loc[0].drop(['click', 'advertiser']).tolist()
if test_sample[3] == 'iOS':
    test_sample = xgb.DMatrix([hashnum(modnum, test_sample[0]), hashnum(modnum, test_sample[2]), 0])
else:
    test_sample = xgb.DMatrix([hashnum(modnum, test_sample[0]), hashnum(modnum, test_sample[2]), 1])
[float(model.predict(test_sample))for model in xgb_models]

[0.08725995570421219,
 0.14066007733345032,
 0.2051800787448883,
 0.24862919747829437,
 0.30720844864845276,
 0.3691747486591339,
 0.4615916311740875,
 0.48884275555610657,
 0.4998165965080261,
 0.7143255472183228]